In [97]:
import numpy as np 
import pandas as pd # Data Processing 
from sklearn.model_selection import train_test_split # Categorize trainset, validation set 

import torch
import torch.nn as nn 
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset 
import torch.onnx

import glob # File to list
from tqdm import tqdm # Process bar 

from sklearn.preprocessing import MinMaxScaler



In [98]:
class ResidualBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(ResidualBlock, self).__init__()
        self.hidden1 = nn.Linear(hidden_dim, hidden_dim) 
        self.leaky_relu1 = nn.LeakyReLU(negative_slope=0.01)

        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.glu = nn.GLU() # Gated Linear Unit splits feature into half >> ex 128 to 64.

        self.hidden3 = nn.Linear(hidden_dim//2, hidden_dim)
        self.leaky_relu3 = nn.LeakyReLU(negative_slope=0.01)

        self.hidden4 = nn.Linear(hidden_dim, hidden_dim) 
        self.glu2 = nn.GLU() # half

        self.hidden5 = nn.Linear(hidden_dim//2, input_dim) 
        self.leaky_relu5 = nn.LeakyReLU(negative_slope=0.01)

        # 32 x 64 / 128 x 64

    def forward(self, x):
        out = self.leaky_relu1(self.hidden1(x))
        out = self.glu(self.hidden2(out))
        out = self.leaky_relu3(self.hidden3(out))
        out = self.glu2(self.hidden4(out))
        out = self.leaky_relu5(self.hidden5(out))
        return out

class MovePredictionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MovePredictionModel, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        #self.leaky_relu = nn.LeakyReLU(negative_slope=0.3)

        # Residual block
        self.residual_block = ResidualBlock(input_dim, hidden_dim)

        # self.hidden_skiplayer = nn.Linear(hidden_dim//2, input_dim)
        self.hidden_skiplayer = nn.Linear(input_dim, hidden_dim)
        self.leakyrelu_skiplayer = nn.LeakyReLU(negative_slope=0.01)

        #self.hidden = nn.Linear(hidden_dim, output_dim)
        #self.leakyrelu = nn.LeakyReLU(negative_slope=0.02)
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        #processed_input = self.leaky_relu(self.input_layer(x))
        processed_input = self.input_layer(x)
        residual_out = self.residual_block(processed_input) # output tensor size: input_dim
        
        out = x + residual_out
        out = self.hidden_skiplayer(out)
        out = self.leakyrelu_skiplayer(out)
        out = self.output_layer(out)
        return out

In [99]:
INPUT_DIM = 9 # Linear vel + Linear acc + Angular vel + Angular acc + pos diff(vel*delta t) + rot diff(angvel * delta t) + bIsDrifting
HIDDEN_DIM = 128
OUTPUT_DIM = 3 # Pos diff x y  and rot diff z 
EPOCHS = 5000
BATCH_SIZE = 64 # TODO
LEARNING_RATE = 1e-4 #0.0005

OUTPUT_FILENAME = 'model/test.pth'
DATA_PATH = 'data'

cuda_available = torch.cuda.is_available()
print(cuda_available)

True


In [100]:
# Read .csv files with pandas
file_path = "data/data0.csv"
data = pd.read_csv(file_path) # data frame 

# csv_file_paths = glob.glob(f'{DATA_PATH}/**/*.csv', recursive=True)
# for path in csv_file_paths:
#     print(path)

# Convert to tensor 
x_features_csv = [i for i in range(9)] # 0 - 8 idx
y_features_csv = [i for i in range(9, 12)] # 9 - 11 idx 

input_features = data.values[:,x_features_csv] # 0 - 8
output_features = data.values[:, y_features_csv]

# Min max scaling
x_min, x_max = input_features.min(axis=0), input_features.max(axis=0)
y_min, y_max = output_features.min(axis=0), output_features.max(axis=0)

input_features_scaled = (input_features - x_min) / (x_max - x_min)
#output_features_scaled = (output_features - y_min) / (y_max - y_min)

X = torch.tensor(input_features_scaled, dtype=torch.float32)
y = torch.tensor(output_features, dtype=torch.float32)

print(y)
print(torch.mean(y, dim=0))
#print(X.shape, y.shape)

# Dataset split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
 
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# print(f'Training samples: {len(train_loader.dataset)}')
# print(f'Validation samples: {len(val_loader.dataset)}')

batch = next(iter(train_loader))  # 첫 번째 배치 가져오기
print(batch[0].shape) 


tensor([[ 6.7012e+01,  2.9694e+01,  0.0000e+00],
        [ 7.6529e+01,  3.2894e+01, -5.2360e-02],
        [ 8.6817e+01,  3.4000e+01, -5.2360e-02],
        ...,
        [-5.0843e+01, -2.8318e+01,  0.0000e+00],
        [-2.8636e+01, -1.5950e+01,  0.0000e+00],
        [-1.0118e+01, -5.6353e+00,  0.0000e+00]])
tensor([ 1.7037e-01,  1.3411e+00, -3.8174e-04])
torch.Size([64, 9])


In [101]:
# Set model
model = MovePredictionModel(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
#model = ResidualBlock(INPUT_DIM, HIDDEN_DIM)
if cuda_available:
    model.cuda()

epochs = EPOCHS
best_val_rmse = float('inf')
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

loss_weights = torch.tensor([1,1,1e+2]) # Output features average >> [ 1.7037e-01,  1.3411e+00, -3.8174e-04] therefore I multiplied 1e+2 to the 3rd feature
if cuda_available:
    loss_weights = loss_weights.cuda()

# Early stopping 
patience = 20
es_counter = 0
early_stop = False

for epoch in range(epochs):
    # Tranining 
    model.train()
    #for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} Training"):
    for X_batch, y_batch in train_loader:
        if cuda_available:
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
        
        optimizer.zero_grad()

        y_pred = model(X_batch)

        loss = criterion(y_pred * loss_weights, y_batch * loss_weights)
        loss.backward()
        optimizer.step()

    if epoch % 20 == 0 or epoch == epochs-1:

        model.eval()

        val_rmse = []
        y_preds = []
        y_actuals = []
        with torch.no_grad():
            #for X_batch, y_batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} Validation"):
            for X_batch, y_batch in val_loader:

                ## if you have GPU
                if cuda_available:
                    X_batch, y_batch = X_batch.cuda(), y_batch.cuda()

                # inference the model
                y_pred = model(X_batch)

                # calculate RMSE
                rmse = torch.sqrt(criterion(y_pred, y_batch)).cpu().numpy()
                val_rmse.append(rmse)

                # for the first batch
                if len(y_preds) == 0:  
                    y_preds = y_pred.cpu().numpy()
                    y_actuals = y_batch.cpu().numpy()
                # for the rest of the batches
                else:  
                    y_preds = np.vstack((y_preds, y_pred.cpu().numpy()))
                    y_actuals = np.vstack((y_actuals, y_batch.cpu().numpy()))
        epoch_val_rmse = np.mean(val_rmse)
        print(f"Epoch {epoch+1}, Validation RMSE: {epoch_val_rmse}")

        if epoch_val_rmse < best_val_rmse:
            best_val_rmse = epoch_val_rmse
            print(f"New best model with RMSE: {best_val_rmse}, saving model...")
            torch.save(model.state_dict(), OUTPUT_FILENAME)
            es_counter = 0
        else:
            es_counter += 1
            print(f"Validation RMSE did not improve. Patience counter: {es_counter}/{patience}")
            if es_counter >= patience:
                print("Early stopping triggered. End loop")
                early_stop = True
                break
    if early_stop:
        model.load_state_dict(torch.load(output_features-OUTPUT_FILENAME))

Epoch 1, Validation RMSE: 155.19293212890625
New best model with RMSE: 155.19293212890625, saving model...
Epoch 21, Validation RMSE: 117.31716918945312
New best model with RMSE: 117.31716918945312, saving model...
Epoch 41, Validation RMSE: 104.00631713867188
New best model with RMSE: 104.00631713867188, saving model...
Epoch 61, Validation RMSE: 23.268033981323242
New best model with RMSE: 23.268033981323242, saving model...
Epoch 81, Validation RMSE: 17.510108947753906
New best model with RMSE: 17.510108947753906, saving model...
Epoch 101, Validation RMSE: 14.908616065979004
New best model with RMSE: 14.908616065979004, saving model...
Epoch 121, Validation RMSE: 11.775607109069824
New best model with RMSE: 11.775607109069824, saving model...
Epoch 141, Validation RMSE: 11.440333366394043
New best model with RMSE: 11.440333366394043, saving model...
Epoch 161, Validation RMSE: 12.931700706481934
Validation RMSE did not improve. Patience counter: 1/20
Epoch 181, Validation RMSE: 13.

In [102]:
ONNX_PATH = "model/KartPredictionModel.onnx"

model = MovePredictionModel(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
model.load_state_dict(torch.load(OUTPUT_FILENAME))  
model.eval() 

dummy_input = torch.randn(1, INPUT_DIM) 

torch.onnx.export(
    model, 
    dummy_input, 
    ONNX_PATH, 
    verbose=True, 
    input_names=['input'], 
    output_names=['output'], 
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)


C:\Users\01076\AppData\Local\Temp\ipykernel_62660\2314137942.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(OUTPUT_FILENAME))  # pth 파일